In [2]:
import cv2
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import skimage.io as io

In [4]:
def fit_elliptical_boundary_model(features):
    # Fit a Gaussian Mixture Model (GMM) to the features
    gmm = GaussianMixture(n_components=1, covariance_type='full')
    gmm.fit(features) 
    # Get the parameters of the fitted Gaussian component
    mean = gmm.means_[0]
    print(gmm.means_)
    cov_matrix = gmm.covariances_[0]

    # Calculate the eigenvalues and eigenvectors of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sort the eigenvalues and eigenvectors in descending order
    indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[indices]
    eigenvectors = eigenvectors[:, indices]

    # Calculate the angle of rotation for the ellipse
    angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))

    # Create an ellipse using matplotlib
    ellipse = Ellipse(mean, 2 * np.sqrt(5.991 * eigenvalues[0]),
                      2 * np.sqrt(5.991 * eigenvalues[1]), angle=angle,
                      fill=False, color='red', linewidth=2)

    return ellipse


In [5]:
def visualize_elliptical_boundary_model(image, ellipse):
    # Display the image with the fitted ellipse
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.gca().add_patch(ellipse)
    plt.axis('off')
    plt.show()

In [6]:

# Load an example image
# image_path = "face.jpeg"
# image = cv2.imread(image_path)
camera = cv2.VideoCapture(0)
cascade_path = "./data/haarcascades/haarcascade_frontalface_default.xml"
clf = cv2.CascadeClassifier(str(cascade_path))

while True:
    # Get Frames from camera
    _,image = camera.read()
    # Convert the image to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Extract the Cr and Cb channels as features
    features = ycrcb[:, :, 1:3].reshape((-1, 2))

    # Fit an elliptical boundary model to the features
    ellipse = fit_elliptical_boundary_model(features)
    # ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # # Define a color range for skin tones in YCbCr space
    # lower_skin = np.array([0, 133, 77], dtype="uint8")
    # upper_skin = np.array([255, 173, 127], dtype="uint8")

    # # Create a binary mask for skin pixels
    # skin_mask = cv2.inRange(ycbcr, lower_skin, upper_skin)

    # # Apply the mask to the original image
    # segmented_image = cv2.bitwise_and(image, image, mask=skin_mask)

    # Visualize the elliptical boundary model
    # visualize_elliptical_boundary_model(image, ellipse)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = clf.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # rendering the faces
    for (x, y, width, height) in faces:
        cv2.rectangle(image, (x, y), (x+width, y+height), (0, 255, 0), 2)
        image[y:y+height, x:x+width, 0] = gray[y:y+height, x:x+width]
        image[y:y+height, x:x+width, 1] = gray[y:y+height, x:x+width]
        image[y:y+height, x:x+width, 2] = gray[y:y+height, x:x+width]

    cv2.imshow("faces", image)

    if (cv2.waitKey(1) == ord("q")):
        break

camera.release()
cv2.destroyAllWindows()

[[132.59861328 123.13392904]]
[[132.05597331 124.18502604]]
[[132.01810872 124.16430013]]
[[132.10290039 123.97375651]]
[[132.11009115 123.96768555]]
[[132.01414063 123.93054362]]
[[132.33209635 123.53717773]]
[[132.25957682 123.50180664]]
[[132.16727865 123.38716797]]
[[132.21319336 123.59216146]]
[[132.26062826 123.41244792]]
[[131.98709635 123.54932617]]
[[132.11166667 123.44059245]]
[[132.07909505 123.46219076]]
[[132.11976888 123.45639974]]
[[132.09439128 123.42622721]]
[[132.09539062 123.48698242]]
[[132.18539388 123.38213542]]
[[132.25073568 123.35658203]]
[[132.34635091 123.18013021]]
[[132.47614583 123.14063151]]
[[132.32138997 123.19088542]]
[[132.35214844 123.27225911]]
[[132.33509766 123.33300781]]
[[132.3384082  123.31283203]]
[[132.29684245 123.33046875]]
[[132.40196289 123.23504232]]
[[132.35927409 123.24586589]]
[[132.36750326 123.27525065]]
[[132.37877279 123.28541341]]
[[132.33999349 123.25887695]]
[[132.32332682 123.27005208]]
[[132.21886393 123.51962891]]
[[132.2420

In [ ]:
cascade_path = "./data/haarcascades/haarcascade_frontalface_default.xml"

print(cascade_path)

# make classifier

clf = cv2.CascadeClassifier(str(cascade_path))

# defining camera footage
camera = cv2.VideoCapture(0)


# render loop

while True:
    # reading form camera and detecting faces
    _, frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = clf.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # rendering the faces
    for (x, y, width, height) in faces:
        cv2.rectangle(frame, (x, y), (x+width, y+height), (0, 255, 0), 2)
        frame[y:y+height, x:x+width, 0] = gray[y:y+height, x:x+width]
        frame[y:y+height, x:x+width, 1] = gray[y:y+height, x:x+width]
        frame[y:y+height, x:x+width, 2] = gray[y:y+height, x:x+width]

    cv2.imshow("faces", frame)

    if (cv2.waitKey(1) == ord("q")):
        break

camera.release()
cv2.destroyAllWindows()